# Can intracellular TE models work with extracellular samples?
```
6/5/23
Andrew E. Davidson
aedavids@ucsc.edu
```


## <span style="color:red;background-color:yellow">AEDWIP TODO loadTx2Genes() finds repeat!</span>
3/21/24 All TE's are repeats not all repeats are TE's

replace loadTx2Genes()

```
import intraExtraRNA.transposableElementUtilities as teu
tx2GeneDF, geneCodeGeneGeneIdList, transposableElementGeneIdList = teu.loadTx2Genes(tx2geneFilePath)
```

with cell [25] and [28] in extraCellularRNA/terra/GTExTCGA_Release/jupyterNotebooks/boxPlots.ipynb
getTEGenesIdx() and createCladeDict()
<hr />

ref:  
extraCellularRNA/intraExtraRNA_POC/jupyterNotebooks/bestSignatureGenesBaseMeanBugFixPOC.ipynb
    
**todo**
<span style="color:red"> model trained on intracellular did not work on extra. Unlikely model trained on intra TE will work.</span>


In [1]:
from IPython.display import display 
import joblib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib as pl
from pprint import pprint
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression

# add our local python packages to path
import sys
sys.path.append('../python/src')
from confusion_matrix.cf_matrix import make_confusion_matrix
import intraExtraRNA.deseqUtilities as du
import intraExtraRNA.mapIds as mi
import intraExtraRNA.transposableElementUtilities as teu
import intraExtraRNA.utilities as ut

LOCAL_CACHE_DIR="/scratch/aedavids/tmp"

NOTEBOOK_NAME = "testIntraCellular-TE-ModelOnExtracellularSamples"
imgOutPl = pl.Path("img").joinpath(NOTEBOOK_NAME)
imgOutPl.mkdir(parents=True, exist_ok=True)

In [3]:
# load the trained model
modelOutPL = pl.Path("models").joinpath("testIntraCellular-TE-Biomarkers")
modelFilePath = modelOutPL.joinpath("logisticRegresion.data")
lrModel = joblib.load(modelFilePath)
print(f'lrModel.coef_\n{lrModel.coef_}')
print(f'\nlrModel.intercept_\n{lrModel.intercept_}')

# get the list of features
features = lrModel.features
print(f'\nlrModel.features\n{features}')

aedwip load standards scaler

lrModel.coef_
[[-1.89594324 -2.57107744 -0.82617097  0.27063084 -1.17277265 -1.61790618
  -0.37738312 -0.16954846  1.45560711 -0.11033368  0.21284962 -0.9403839
   0.92417859 -0.34984218  0.65316851 -0.06789157 -0.24509306 -0.4822364 ]]

lrModel.intercept_
[-0.53124101]

lrModel.features
['MER65-int', 'MER65C', 'LTR54B', 'LTR81B', 'MER65B', '(GGT)n', 'HERV30-int', '(TTCTCG)n', '(TGAA)n', '(TTCAT)n', '(CCG)n', '(ACCAC)n', '(TACACA)n', '(GCC)n', '(GAGGC)n', '(GCG)n', '(CGG)n', '(CTCCCT)n']


## Check features Transposable elements

In [4]:
%%time
# countDF gene ids are in in ENSG format or loci . ex. ENSG00000227232.5 and (AAAAAC)n
file="/private/groups/kimlab/genomes.annotations/gencode.35/gencode.v35.ucsc.rmsk.tx.to.gene.csv"
tx2GenePath = groupByGeneCountFilePath = ut.load(file, localCacheDir=LOCAL_CACHE_DIR, verbose=True)
tx2GeneDF, geneCodeGeneGeneIdList, transposableElementGeneIdList = teu.loadTx2Genes(tx2GenePath)

print( set(features).intersection( set(transposableElementGeneIdList ) ))

localTargetPath:
/scratch/aedavids/tmp/private/groups/kimlab/genomes.annotations/gencode.35/gencode.v35.ucsc.rmsk.tx.to.gene.csv

{'(GCC)n', '(CGG)n', 'MER65C', '(GAGGC)n', '(TTCTCG)n', '(TTCAT)n', '(CTCCCT)n', 'MER65-int', '(TACACA)n', '(GCG)n', 'MER65B', '(ACCAC)n', 'HERV30-int', '(GGT)n', '(CCG)n', 'LTR81B', 'LTR54B', '(TGAA)n'}
CPU times: user 18.8 s, sys: 2.41 s, total: 21.2 s
Wall time: 21.2 s


## deprecated Map best TE and repeats gene name to ensembl ENSG format

In [ ]:

# # features geneId HUGO format.
# ENSGFeatureDF = mi.mapHUGO_2_ENSG(file, features)
# print(ENSGFeatureDF.shape)
# ENSGFeatureDF

In [ ]:
# ENSGFeatureList = ENSGFeatureDF.loc[:, "ENSG"].tolist()

In [ ]:
# # bestGeneENSGList = list( bestGeneENSGDF.loc[:, "ENSG"].unique() )
# print(' '.join(ENSGFeatureList[0:5]))
# print(' '.join(ENSGFeatureList[5:10]))
# print(' '.join(ENSGFeatureList[10:15]))
# print(' '.join(ENSGFeatureList[15:20]))
# print(' '.join(ENSGFeatureList[20:]))

## Load count data

In [5]:
rootDir   = '/private/groups/kimlab/aedavids_data'

countDataPath = rootDir + "/" + 'panc_normal_norm_counts_2023-05-18.csv'
countDataPath = ut.load(countDataPath, localCacheDir=LOCAL_CACHE_DIR, verbose=True)
countDF = pd.read_csv(countDataPath, index_col='gene')
print(f'countDF.shape : {countDF.shape}')

metaDataPath  = rootDir + "/" +'panc_normal_scaled_metaData_2023-05-18.csv'
metaDataPath = ut.load(metaDataPath, localCacheDir=LOCAL_CACHE_DIR, verbose=True)
metaDF = pd.read_csv(metaDataPath)
print(f'metaDF.shape : {metaDF.shape}')

localTargetPath:
/scratch/aedavids/tmp/private/groups/kimlab/aedavids_data/panc_normal_norm_counts_2023-05-18.csv

countDF.shape : (76555, 32)
localTargetPath:
/scratch/aedavids/tmp/private/groups/kimlab/aedavids_data/panc_normal_scaled_metaData_2023-05-18.csv

metaDF.shape : (32, 17)


In [6]:
countDF.head()

,panc.1.2.3,panc.2.2.7,panc.3.2.5,panc.4.3.9,panc.5.4.5,panc.8.2.5,panc.7.1.1,panc.9.1.1,panc.10.1.2,panc.6.1.1,...,n31,n32,n33,n34,n35,n41,n42,n43,n44,n45
gene,,,,,,,,,,,,,,,,,,,,,
(A)n,0.0,0.0,1.398199,2.782073,0.0,0.448711,1.496324,10.788154,0.74722,1.685168,...,160.867843,120.270349,174.764405,147.27735,204.488252,109.839352,104.30979,379.943132,445.9268,139.999428
(AAA)n,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000
(AAAAAAC)n,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000
(AAAAAAG)n,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000
(AAAAAAT)n,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000


##  depricated we get an error when we select the Best ENSG values from count
```
missing: Index(['ENSG00000148053.16', 'ENSG00000275395.6', 'ENSG00000103034.14',
                'ENSG00000079215.14', 'ENSG00000164266.10']
```

add them to count data with a value of zero

In [ ]:
# missingENSGGenesList = ['ENSG00000148053.16' 
#                         ,'ENSG00000275395.6' 
#                         ,'ENSG00000103034.14'
#                         ,'ENSG00000079215.14' 
#                         ,'ENSG00000164266.10'
#                        ]

# # mapENSG_2_HUGO() is slow. however we are garuanteed the ordering is correct 
# missingHUGOGenesDF = mi.mapENSG_2_HUGO(tx2GenePath, missingENSGGenesList)
# print("missing HUGO genes")
# display(missingHUGOGenesDF)

# numberOfSamples = countDF.shape[1] 
# zeros = np.zeros( numberOfSamples )
# for missingFeature in missingENSGGenesList:
#     # add the missing row
#     countDF.loc[missingFeature] = zeros

In [ ]:
# # make sure the rows order matches the model's feature order
# countDF['HUGO'] = features
# countDF = countDF.set_index('HUGO')
# countDF.reindex()
# # print()
# # print(countDF.columns)
# # display( countDF )

# # re-order
# countDF = countDF.loc[features]
# countDF

In [10]:
countDF = countDF.loc[features, :]
print(countDF.shape)
countDF.head()

(18, 32)


,panc.1.2.3,panc.2.2.7,panc.3.2.5,panc.4.3.9,panc.5.4.5,panc.8.2.5,panc.7.1.1,panc.9.1.1,panc.10.1.2,panc.6.1.1,...,n31,n32,n33,n34,n35,n41,n42,n43,n44,n45
gene,,,,,,,,,,,,,,,,,,,,,
MER65-int,0.0,25.528217,15.380191,19.474514,71.470476,17.948435,2.493873,14.159452,1.120830,0.561723,...,0.00000,0.595398,3.971918,5.522901,2.429563,0.000000,0.00000,35.797640,9.564114,2.285705
MER65C,0.0,0.000000,0.000000,0.000000,0.000000,4.487109,6.982844,8.765375,0.747220,2.246891,...,1.38084,4.167784,0.000000,0.613656,2.024636,5.089082,0.00000,0.813583,0.000000,9.142820
LTR54B,0.0,12.764109,1.398199,2.782073,14.294095,0.897422,9.476717,5.394077,1.868049,5.055504,...,3.45210,4.167784,0.000000,6.136556,4.049272,0.424090,3.67288,0.000000,11.955142,0.000000
LTR81B,0.0,0.000000,1.398199,0.000000,0.000000,2.243554,0.000000,0.674260,0.747220,2.246891,...,0.00000,0.595398,0.661986,1.840967,2.024636,0.000000,0.00000,1.627165,0.000000,0.571426
MER65B,0.0,0.000000,0.000000,0.000000,0.000000,0.448711,0.000000,1.348519,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.142852


## Run logistic regression

In [11]:
conditionList = metaDF.loc[:, 'condition'].to_list()

labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(conditionList)

# quick visual check
pd.DataFrame( {
    'cat':conditionList,
    'oneHot': y } )

display(tmpDF.groupby(by="cat").count())
display(tmpDF.head())
print()
tmpDF.tail()

In [ ]:
XNP = countDF.values.transpose()
print(XNP.shape)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# lrModel = LogisticRegression()
#             , penalty='l2' # regularization some penalities do not work with some solvers
# tol=0.0001
# lrModel.fit(XNP, y, C=1.0 
#             ,random_state=42           
#             ,solver='lbfgs' 
#             , max_iterint=100
#           )

#scaler = StandardScaler().fit(XNP)
XScaled = scaler.transform(XNP)

#lrModel = lrModel.fit(XScaled, y)

In [ ]:
yHat = lrModel.predict(XScaled)
print('{} Accuracy : {:.2f}'.format(lrModel, lrModel.score(XScaled, y)))

In [ ]:
confusion_matrix = confusion_matrix(y, yHat)
print(confusion_matrix)

In [ ]:
labels = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
# categories = ['Zero', 'One']
categories = ['PAAD', 'Pancrease']
aedwip check categories
#aedwip title =  "Extracellular Pancrease & PAAD. Intracellular PAAD 1vsAll biomarkers"
title = "Predictions made for extracelluar Pancrease & PAAD samples by a model trained on intracellular RNA"
fig, panel = make_confusion_matrix(confusion_matrix, 
                      # group_names=labels,
                      categories=categories, 
                      cbar=False,
                      count=True,
                      cmap='Blues',
                      title=title
                     )

In [ ]:
imgFilePL = imgOutPl.joinpath(NOTEBOOK_NAME + "-ConfusionMatrix.png")
fig.savefig(imgFilePL, dpi=600,  bbox_inches='tight', facecolor="white", transparent=True )